In [ ]:
"""
Windows FGDB to SQL Server Converter
Optimized for Windows environment with direct SQL Server access
"""

import geopandas as gpd
import fiona
import pyodbc
from sqlalchemy import create_engine, text
import os
import pandas as pd
from pathlib import Path

class WindowsFGDBConverter:
    def __init__(self, server, database, fgdb_path):
        self.server = server
        self.database = database
        self.fgdb_path = fgdb_path
        self.engine = None

    def create_connection(self):
        """Create SQL Server connection using Windows authentication"""
        try:
            # SQLAlchemy connection string for Windows auth
            connection_string = f"mssql+pyodbc://@{self.server}/{self.database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

            self.engine = create_engine(connection_string, fast_executemany=True)

            # Test connection
            with self.engine.connect() as conn:
                result = conn.execute(text("SELECT @@VERSION"))
                version = result.fetchone()[0]
                print(f"✅ Connected to SQL Server")
                print(f"   Version: {version[:60]}...")
                return True

        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False

    def list_fgdb_layers(self):
        """List all layers in the FGDB"""
        try:
            if not os.path.exists(self.fgdb_path):
                print(f"❌ FGDB not found: {self.fgdb_path}")
                return []

            layers = fiona.listlayers(self.fgdb_path)
            print(f"📋 Found {len(layers)} layers in FGDB:")
            for i, layer in enumerate(layers, 1):
                print(f"   {i}. {layer}")

            return layers

        except Exception as e:
            print(f"❌ Error listing FGDB layers: {e}")
            return []

    def get_layer_preview(self, layer_name, sample_size=5):
        """Get a preview of the layer data"""
        try:
            gdf = gpd.read_file(self.fgdb_path, layer=layer_name, rows=sample_size)

            info = {
                'total_features': len(gpd.read_file(self.fgdb_path, layer=layer_name, rows=1000).index),  # Quick count
                'geometry_type': gdf.geometry.geom_type.mode().iloc[0] if len(gdf) > 0 else 'Unknown',
                'columns': list(gdf.columns),
                'crs': gdf.crs.to_string() if gdf.crs else 'Unknown',
                'bounds': gdf.total_bounds if len(gdf) > 0 else None
            }

            return info, gdf.head(sample_size)

        except Exception as e:
            print(f"   ⚠️ Error getting preview: {e}")
            return None, None

    def clean_column_names(self, columns):
        """Clean column names for SQL Server compatibility"""
        cleaned = []
        for col in columns:
            # Replace problematic characters
            clean_col = col.replace(' ', '_').replace('-', '_').replace('.', '_')
            clean_col = clean_col.replace('(', '').replace(')', '')
            clean_col = clean_col.replace('#', 'num').replace('%', 'pct')

            # Ensure it starts with a letter
            if clean_col[0].isdigit():
                clean_col = f"col_{clean_col}"

            # Limit length
            if len(clean_col) > 50:
                clean_col = clean_col[:50]

            cleaned.append(clean_col)

        return cleaned

    def determine_sql_types(self, gdf):
        """Determine appropriate SQL Server data types"""
        dtype_mapping = {}

        for col in gdf.columns:
            if col == 'geometry':
                continue

            if gdf[col].dtype == 'object':
                # String column - check max length
                max_len = gdf[col].astype(str).str.len().max()
                if pd.isna(max_len) or max_len == 0:
                    dtype_mapping[col] = 'NVARCHAR(100)'
                elif max_len > 4000:
                    dtype_mapping[col] = 'TEXT'
                else:
                    dtype_mapping[col] = f'NVARCHAR({min(int(max_len * 1.5), 4000)})'

            elif gdf[col].dtype in ['int64', 'int32', 'int16']:
                dtype_mapping[col] = 'INTEGER'

            elif gdf[col].dtype in ['float64', 'float32']:
                dtype_mapping[col] = 'FLOAT'

            elif gdf[col].dtype == 'bool':
                dtype_mapping[col] = 'BIT'

            else:
                dtype_mapping[col] = 'NVARCHAR(255)'  # Default

        # Add geometry column
        dtype_mapping['Shape'] = 'GEOMETRY'

        return dtype_mapping

    def convert_layer(self, layer_name, chunk_size=1000):
        """Convert a single layer from FGDB to SQL Server"""
        print(f"\n🔄 Converting layer: {layer_name}")

        try:
            # Get layer preview
            info, preview = self.get_layer_preview(layer_name)
            if info:
                print(f"   📊 Features: ~{info['total_features']}")
                print(f"   🗺️ Geometry: {info['geometry_type']}")
                print(f"   📋 Columns: {len(info['columns'])}")
                print(f"   🌐 CRS: {info['crs']}")

            # Read full layer
            print(f"   📖 Reading layer data...")
            gdf = gpd.read_file(self.fgdb_path, layer=layer_name)

            if len(gdf) == 0:
                print(f"   ⚠️ Layer is empty, skipping")
                return True

            # Clean column names
            original_columns = list(gdf.columns)
            cleaned_columns = self.clean_column_names(original_columns)
            column_mapping = dict(zip(original_columns, cleaned_columns))
            gdf = gdf.rename(columns=column_mapping)

            # Handle geometry
            if 'geometry' in gdf.columns:
                print(f"   🔧 Converting geometry to WKT...")
                gdf['Shape'] = gdf['geometry'].apply(lambda x: x.wkt if x and x.is_valid else None)
                gdf = gdf.drop('geometry', axis=1)

            # Determine data types
            dtype_mapping = self.determine_sql_types(gdf)

            print(f"   💾 Writing {len(gdf)} records to SQL Server...")

            # Drop existing table if it exists
            with self.engine.connect() as conn:
                conn.execute(text(f"IF OBJECT_ID('{layer_name}', 'U') IS NOT NULL DROP TABLE {layer_name}"))
                conn.commit()

            # Write data in chunks
            if len(gdf) > chunk_size:
                print(f"   📦 Writing in chunks of {chunk_size}...")

            gdf.to_sql(
                name=layer_name,
                con=self.engine,
                if_exists='replace',
                index=False,
                dtype=dtype_mapping,
                chunksize=chunk_size,
                method='multi'
            )

            # Create spatial index
            print(f"   🗂️ Creating spatial index...")

            # Calculate bounding box from data
            if info and info['bounds'] is not None:
                bounds = info['bounds']
                buffer = 0.1  # Add small buffer
                bbox = f"({bounds[0]-buffer}, {bounds[1]-buffer}, {bounds[2]+buffer}, {bounds[3]+buffer})"
            else:
                bbox = "(-180, -90, 180, 90)"  # World extent as fallback

            spatial_index_sql = f"""
            CREATE SPATIAL INDEX SIDX_{layer_name}_Shape
            ON {layer_name}(Shape)
            USING GEOMETRY_GRID
            WITH (BOUNDING_BOX = {bbox})
            """

            with self.engine.connect() as conn:
                conn.execute(text(spatial_index_sql))
                conn.commit()

            print(f"   ✅ Successfully converted {layer_name}")

            # Show column mapping if names were changed
            changed_cols = [(orig, new) for orig, new in column_mapping.items() if orig != new and orig != 'geometry']
            if changed_cols:
                print(f"   📝 Column name changes:")
                for orig, new in changed_cols[:5]:  # Show first 5
                    print(f"      {orig} → {new}")
                if len(changed_cols) > 5:
                    print(f"      ... and {len(changed_cols) - 5} more")

            return True

        except Exception as e:
            print(f"   ❌ Error converting {layer_name}: {e}")
            return False

    def convert_all_layers(self, selected_layers=None):
        """Convert all or selected layers"""
        print("🚀 Starting FGDB to SQL Server conversion")
        print(f"📁 Source: {self.fgdb_path}")
        print(f"🎯 Target: {self.server} -> {self.database}")

        # Test connection
        if not self.create_connection():
            return False

        # Get layers
        available_layers = self.list_fgdb_layers()
        if not available_layers:
            return False

        # Determine which layers to convert
        if selected_layers:
            layers_to_convert = [l for l in selected_layers if l in available_layers]
            print(f"\n🎯 Converting {len(layers_to_convert)} selected layers")
        else:
            layers_to_convert = available_layers
            print(f"\n🎯 Converting all {len(layers_to_convert)} layers")

        # Convert each layer
        successful = 0
        failed = 0

        for i, layer in enumerate(layers_to_convert, 1):
            print(f"\n[{i}/{len(layers_to_convert)}] Processing: {layer}")

            if self.convert_layer(layer):
                successful += 1
            else:
                failed += 1

        # Summary
        print(f"\n📊 Conversion Summary:")
        print(f"   ✅ Successful: {successful}")
        print(f"   ❌ Failed: {failed}")
        print(f"   📋 Total: {len(layers_to_convert)}")

        if successful > 0:
            print(f"\n🎉 Conversion complete! You can now query your data:")
            print(f"   Example: SELECT COUNT(*) FROM {layers_to_convert[0]};")
            print(f"   Geometry: SELECT Name, Shape.STAsText() FROM {layers_to_convert[0]};")

        return failed == 0

def main():
    """Main function"""
    # Configuration - UPDATE THESE VALUES
    CONFIG = {
        'server': '100.103.17.32,1433',
        'database': 'SpatialTest',
        'fgdb_path': r'D:\arcgis\clients\sample.gdb'  # UPDATE THIS PATH
    }

    print("=== Windows FGDB to SQL Server Converter ===")

    # Validate FGDB path
    if not os.path.exists(CONFIG['fgdb_path']):
        print(f"❌ FGDB not found: {CONFIG['fgdb_path']}")
        print("Please update the fgdb_path in the CONFIG section")

        # Try to find common locations
        possible_paths = [
            r'D:\arcgis\clients',
            r'C:\arcgis\clients',
            r'D:\data',
            r'C:\data'
        ]

        print("\nLooking for .gdb files in common locations:")
        for path in possible_paths:
            if os.path.exists(path):
                gdb_files = list(Path(path).glob('*.gdb'))
                if gdb_files:
                    print(f"   Found .gdb files in {path}:")
                    for gdb in gdb_files[:5]:  # Show first 5
                        print(f"      {gdb}")
        return

    # Create converter and run
    converter = WindowsFGDBConverter(
        server=CONFIG['server'],
        database=CONFIG['database'],
        fgdb_path=CONFIG['fgdb_path']
    )

    # Convert all layers
    success = converter.convert_all_layers()

    # Or convert specific layers:
    # success = converter.convert_all_layers(['states', 'fedlandp'])

    if success:
        print("\n🏆 All conversions completed successfully!")
    else:
        print("\n⚠️ Some conversions failed. Check the output above for details.")

if __name__ == "__main__":
    main()

In [2]:
# FGDB to SQL Server Conversion - Jupyter Notebook Version
# Cell 1: Test Package Imports

print("=== Testing Python Package Imports ===\n")

# Test each package individually
packages_status = {}

try:
    import geopandas as gpd
    print("✅ geopandas imported successfully")
    print(f"   Version: {gpd.__version__}")
    packages_status['geopandas'] = True
except ImportError as e:
    print(f"❌ geopandas import failed: {e}")
    packages_status['geopandas'] = False

try:
    import fiona
    print("✅ fiona imported successfully")
    print(f"   Version: {fiona.__version__}")
    packages_status['fiona'] = True
except ImportError as e:
    print(f"❌ fiona import failed: {e}")
    packages_status['fiona'] = False

try:
    import pyodbc
    print("✅ pyodbc imported successfully")
    print(f"   Version: {pyodbc.version}")
    packages_status['pyodbc'] = True
except ImportError as e:
    print(f"❌ pyodbc import failed: {e}")
    packages_status['pyodbc'] = False

try:
    from sqlalchemy import create_engine, __version__ as sqlalchemy_version
    print("✅ sqlalchemy imported successfully")
    print(f"   Version: {sqlalchemy_version}")
    packages_status['sqlalchemy'] = True
except ImportError as e:
    print(f"❌ sqlalchemy import failed: {e}")
    packages_status['sqlalchemy'] = False

try:
    import pandas as pd
    print("✅ pandas imported successfully")
    print(f"   Version: {pd.__version__}")
    packages_status['pandas'] = True
except ImportError as e:
    print(f"❌ pandas import failed: {e}")
    packages_status['pandas'] = False

print(f"\n📊 Package Summary:")
for pkg, status in packages_status.items():
    status_icon = "✅" if status else "❌"
    print(f"   {status_icon} {pkg}")

all_packages_ok = all(packages_status.values())
print(f"\n🎯 All packages ready: {'Yes' if all_packages_ok else 'No'}")

if not all_packages_ok:
    print("\n🔧 To install missing packages, run in a new cell:")
    print("!conda install geopandas pyodbc sqlalchemy fiona -c conda-forge -y")
    print("# or")
    print("!pip install geopandas pyodbc sqlalchemy fiona")

=== Testing Python Package Imports ===

✅ geopandas imported successfully
   Version: 1.1.1
✅ fiona imported successfully
   Version: 1.10.1
✅ pyodbc imported successfully
   Version: 5.2.0
✅ sqlalchemy imported successfully
   Version: 2.0.41
✅ pandas imported successfully
   Version: 2.3.1

📊 Package Summary:
   ✅ geopandas
   ✅ fiona
   ✅ pyodbc
   ✅ sqlalchemy
   ✅ pandas

🎯 All packages ready: Yes


In [6]:
# FGDB to SQL Server Setup Test - PyCharm Version
# Run this script first to verify everything is working

print("=== FGDB to SQL Server Setup Test ===\n")

def test_imports():
    """Test if all required packages work"""
    print("📦 Testing package imports...")

    try:
        import geopandas as gpd
        print(f"✅ geopandas {gpd.__version__}")
    except ImportError as e:
        print(f"❌ geopandas failed: {e}")
        return False

    try:
        import fiona
        print(f"✅ fiona {fiona.__version__}")
    except ImportError as e:
        print(f"❌ fiona failed: {e}")
        return False

    try:
        import pyodbc
        print(f"✅ pyodbc {pyodbc.version}")
    except ImportError as e:
        print(f"❌ pyodbc failed: {e}")
        return False

    try:
        from sqlalchemy import create_engine, __version__ as sqlalchemy_version
        print(f"✅ sqlalchemy {sqlalchemy_version}")
    except ImportError as e:
        print(f"❌ sqlalchemy failed: {e}")
        return False

    try:
        import pandas as pd
        print(f"✅ pandas {pd.__version__}")
    except ImportError as e:
        print(f"❌ pandas failed: {e}")
        return False

    return True

def find_fgdb_files():
    """Look for FGDB files in common locations"""
    import os
    from pathlib import Path

    print("\n🔍 Searching for .gdb files...")

    search_paths = [
        r'D:\arcgis\clients',
        r'C:\arcgis\clients',
        r'D:\data',
        r'C:\data',
        r'D:\arcgis',
        r'C:\arcgis'
    ]

    found_gdbs = []

    for search_path in search_paths:
        if os.path.exists(search_path):
            print(f"   Checking {search_path}...")
            gdb_files = list(Path(search_path).glob('**/*.gdb'))
            if gdb_files:
                print(f"   📁 Found {len(gdb_files)} .gdb files:")
                for gdb in gdb_files[:5]:  # Show first 5
                    print(f"      {gdb}")
                    found_gdbs.append(str(gdb))
                if len(gdb_files) > 5:
                    print(f"      ... and {len(gdb_files) - 5} more")

    if not found_gdbs:
        print("   ❌ No .gdb files found in common locations")
        print("   💡 Try looking in your specific data directories")

    return found_gdbs

def test_fgdb_access(fgdb_path):
    """Test access to a specific FGDB"""
    import os
    import fiona

    print(f"\n📂 Testing FGDB: {fgdb_path}")

    if not os.path.exists(fgdb_path):
        print(f"   ❌ Path does not exist")
        return False

    try:
        layers = fiona.listlayers(fgdb_path)
        print(f"   ✅ Accessible - {len(layers)} layers found:")
        for i, layer in enumerate(layers, 1):
            print(f"      {i}. {layer}")
        return layers
    except Exception as e:
        print(f"   ❌ Error accessing FGDB: {e}")
        return False

def test_sql_server():
    """Test SQL Server connection"""
    import pyodbc

    print(f"\n🔌 Testing SQL Server connection...")

    server = "100.103.17.32,1433"
    database = "SpatialTest"
    username = "dbeaver"
    password = "dbeaver"

    try:
        conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};"
        conn = pyodbc.connect(conn_str, timeout=10)
        cursor = conn.cursor()

        cursor.execute("SELECT @@VERSION")
        version = cursor.fetchone()[0]
        print(f"   ✅ Connected to SQL Server")
        print(f"   📊 Version: {version[:60]}...")

        cursor.execute("SELECT DB_NAME()")
        db_name = cursor.fetchone()[0]
        print(f"   💾 Database: {db_name}")

        conn.close()
        return True

    except Exception as e:
        print(f"   ❌ Connection failed: {e}")
        print(f"   💡 Common issues:")
        print(f"      - SQL Server not running")
        print(f"      - Firewall blocking connection")
        print(f"      - Windows authentication not enabled")
        return False

def main():
    """Run all tests"""

    # Test 1: Package imports
    if not test_imports():
        print("\n❌ Package import test failed!")
        return

    # Test 2: Find FGDB files
    found_gdbs = find_fgdb_files()

    # Test 3: Test specific FGDB if found
    if found_gdbs:
        test_fgdb = found_gdbs[0]  # Test first one found
        layers = test_fgdb_access(test_fgdb)
    else:
        # Try common test path
        test_fgdb = r"Z:\Users\brendanhall\GitHub\General_Code\ATFS\FGDB to SQL Server Conversion\esri_ref_data.gdb\esri_ref_data.gdb"
        layers = test_fgdb_access(test_fgdb)

    # Test 4: SQL Server connection
    sql_ok = test_sql_server()

    # Summary
    print(f"\n" + "="*50)
    print("📋 TEST SUMMARY")
    print("="*50)
    print(f"✅ Python packages: Ready")
    print(f"{'✅' if found_gdbs or layers else '❌'} FGDB access: {'Ready' if found_gdbs or layers else 'Check paths'}")
    print(f"{'✅' if sql_ok else '❌'} SQL Server: {'Ready' if sql_ok else 'Check connection'}")

    if (found_gdbs or layers) and sql_ok:
        print(f"\n🎉 ALL TESTS PASSED! Ready for FGDB conversion.")
        print(f"\n📝 Next steps:")
        print(f"   1. Update CONFIG in the main script with your FGDB path:")
        if found_gdbs:
            print(f"      'fgdb_path': r'{found_gdbs[0]}'")
        print(f"   2. Run the main conversion script")
    else:
        print(f"\n⚠️ Some tests failed. Please resolve issues before conversion.")

if __name__ == "__main__":
    main()

=== FGDB to SQL Server Setup Test ===

📦 Testing package imports...
✅ geopandas 1.1.1
✅ fiona 1.10.1
✅ pyodbc 5.2.0
✅ sqlalchemy 2.0.41
✅ pandas 2.3.1

🔍 Searching for .gdb files...
   ❌ No .gdb files found in common locations
   💡 Try looking in your specific data directories

📂 Testing FGDB: Z:\Users\brendanhall\GitHub\General_Code\ATFS\FGDB to SQL Server Conversion\esri_ref_data.gdb\esri_ref_data.gdb
   ✅ Accessible - 4 layers found:
      1. states
      2. fedlandp
      3. park_dtl
      4. dtl_st

🔌 Testing SQL Server connection...
   ✅ Connected to SQL Server
   📊 Version: Microsoft SQL Server 2022 (RTM-GDR) (KB5046861) - 16.0.1135....
   💾 Database: SpatialTest

📋 TEST SUMMARY
✅ Python packages: Ready
✅ FGDB access: Ready
✅ SQL Server: Ready

🎉 ALL TESTS PASSED! Ready for FGDB conversion.

📝 Next steps:
   1. Update CONFIG in the main script with your FGDB path:
   2. Run the main conversion script


=== FGDB to SQL Server Setup Test ===

📦 Testing package imports...
✅ geopandas 1.1.1
✅ fiona 1.10.1
✅ pyodbc 5.2.0
✅ sqlalchemy 2.0.41
✅ pandas 2.3.1

🔍 Searching for .gdb files...
   ❌ No .gdb files found in common locations
   💡 Try looking in your specific data directories

📂 Testing FGDB: D:\arcgis\clients\sample.gdb
   ❌ Path does not exist

🔌 Testing SQL Server connection...
   ❌ Connection failed: ('28000', '[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed. The login is from an untrusted domain and cannot be used with Integrated authentication. (18452) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed. The login is from an untrusted domain and cannot be used with Integrated authentication. (18452)')
   💡 Common issues:
      - SQL Server not running
      - Firewall blocking connection
      - Windows authentication not enabled

📋 TEST SUMMARY
✅ Python packages: Ready
❌ FGDB access: Check paths
❌ SQL Server:

In [10]:
"""
FGDB to SQL Server Converter - FIXED VERSION
Complete conversion script for ESRI File Geodatabase to SQL Server
"""

import geopandas as gpd
import fiona
import pyodbc
from sqlalchemy import create_engine, text
import pandas as pd
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# CONFIGURATION - UPDATE THESE VALUES
# =============================================================================

CONFIG = {
    'server': '100.103.17.32,1433',
    'database': 'SpatialTest',
    'username': 'dbeaver',
    'password': 'dbeaver',
    'fgdb_path': r'Z:\Users\brendanhall\GitHub\General_Code\ATFS\FGDB to SQL Server Conversion\esri_ref_data.gdb\esri_ref_data.gdb',
    'chunk_size': 1000,
    'selected_layers': None  # None = all layers, or ['layer1', 'layer2'] for specific layers
}

class FGDBConverter:
    def __init__(self, config):
        self.config = config
        self.engine = None

    def log(self, message, level="INFO"):
        """Simple logging function"""
        print(f"[{level}] {message}")

    def setup_connection(self):
        """Create SQL Server connection"""
        self.log("Setting up SQL Server connection...")

        try:
            connection_string = (
                f"mssql+pyodbc://{self.config['username']}:{self.config['password']}"
                f"@{self.config['server']}/{self.config['database']}"
                f"?driver=ODBC+Driver+17+for+SQL+Server"
            )

            self.engine = create_engine(
                connection_string,
                fast_executemany=True,
                pool_pre_ping=True
            )

            # Test connection
            with self.engine.connect() as conn:
                result = conn.execute(text("SELECT @@VERSION, DB_NAME()"))
                version, db_name = result.fetchone()
                self.log(f"✅ Connected to SQL Server")
                self.log(f"   Database: {db_name}")
                self.log(f"   Version: {version[:50]}...")

            return True

        except Exception as e:
            self.log(f"❌ Connection failed: {e}", "ERROR")
            return False

    def get_fgdb_layers(self):
        """Get list of layers in FGDB"""
        self.log(f"Reading FGDB: {self.config['fgdb_path']}")

        if not os.path.exists(self.config['fgdb_path']):
            self.log(f"❌ FGDB not found: {self.config['fgdb_path']}", "ERROR")
            return []

        try:
            layers = fiona.listlayers(self.config['fgdb_path'])
            self.log(f"✅ Found {len(layers)} layers: {', '.join(layers)}")
            return layers

        except Exception as e:
            self.log(f"❌ Error reading FGDB: {e}", "ERROR")
            return []

    def clean_column_names(self, columns):
        """Clean column names for SQL Server compatibility"""
        cleaned = []
        for col in columns:
            # Replace problematic characters
            clean_col = col.replace(' ', '_').replace('-', '_').replace('.', '_')
            clean_col = clean_col.replace('(', '').replace(')', '')
            clean_col = clean_col.replace('#', 'num').replace('%', 'pct')
            clean_col = clean_col.replace('/', '_').replace('\\', '_')

            # Ensure starts with letter
            if clean_col and clean_col[0].isdigit():
                clean_col = f"col_{clean_col}"

            # Limit length
            if len(clean_col) > 50:
                clean_col = clean_col[:50]

            # Handle empty names
            if not clean_col:
                clean_col = f"col_{len(cleaned)}"

            cleaned.append(clean_col)

        return cleaned

    def convert_layer(self, layer_name):
        """FIXED: Simplified conversion that lets pandas handle data types"""
        self.log(f"\n🔄 Converting layer: {layer_name}")

        try:
            # Read the data
            gdf = gpd.read_file(self.config['fgdb_path'], layer=layer_name)

            if len(gdf) == 0:
                self.log(f"   ⚠️ Layer is empty, skipping")
                return True

            self.log(f"   📊 Processing {len(gdf)} features")

            # Show geometry info
            if 'geometry' in gdf.columns and len(gdf) > 0:
                geom_types = gdf.geometry.geom_type.value_counts()
                self.log(f"   🗺️ Geometry types: {dict(geom_types)}")

            # Clean column names
            original_columns = list(gdf.columns)
            cleaned_columns = self.clean_column_names(original_columns)
            column_mapping = dict(zip(original_columns, cleaned_columns))

            # Handle duplicate column names
            if len(set(cleaned_columns)) != len(cleaned_columns):
                self.log(f"   ⚠️ Warning: Duplicate column names after cleaning", "WARN")
                seen = set()
                unique_cleaned = []
                for col in cleaned_columns:
                    if col in seen:
                        counter = 1
                        new_col = f"{col}_{counter}"
                        while new_col in seen:
                            counter += 1
                            new_col = f"{col}_{counter}"
                        unique_cleaned.append(new_col)
                        seen.add(new_col)
                    else:
                        unique_cleaned.append(col)
                        seen.add(col)
                cleaned_columns = unique_cleaned
                column_mapping = dict(zip(original_columns, cleaned_columns))

            gdf = gdf.rename(columns=column_mapping)

            # Handle geometry
            has_geometry = False
            if 'geometry' in gdf.columns:
                self.log(f"   🔧 Converting geometry to WKT...")

                def safe_wkt(geom):
                    try:
                        if geom is None or geom.is_empty:
                            return None
                        if not geom.is_valid:
                            geom = geom.buffer(0)  # Try to fix invalid geometry
                        return geom.wkt
                    except:
                        return None

                gdf['Shape'] = gdf['geometry'].apply(safe_wkt)
                gdf = gdf.drop('geometry', axis=1)
                has_geometry = True

                valid_geoms = gdf['Shape'].notna().sum()
                self.log(f"   ✅ Converted {valid_geoms}/{len(gdf)} geometries to WKT")

            # Drop existing table
            self.log(f"   🗑️ Dropping existing table if exists...")
            with self.engine.connect() as conn:
                conn.execute(text(f"IF OBJECT_ID('{layer_name}', 'U') IS NOT NULL DROP TABLE {layer_name}"))
                conn.commit()

            # Write without specifying dtypes - let pandas decide
            self.log(f"   💾 Writing {len(gdf)} records to SQL Server...")
            gdf.to_sql(
                name=layer_name,
                con=self.engine,
                if_exists='replace',
                index=False,
                chunksize=self.config['chunk_size']
                # No dtype parameter - let pandas auto-detect
            )

            # Fix geometry column type manually if we have geometry
            if has_geometry:
                self.log(f"   🔧 Converting Shape column to GEOMETRY type...")
                try:
                    with self.engine.connect() as conn:
                        # Add a new geometry column
                        conn.execute(text(f"ALTER TABLE {layer_name} ADD Shape_Geom GEOMETRY"))

                        # Update with geometry data
                        conn.execute(text(f"""
                            UPDATE {layer_name}
                            SET Shape_Geom = geometry::STGeomFromText(Shape, 4326)
                            WHERE Shape IS NOT NULL AND Shape != ''
                        """))

                        # Drop old text column and rename
                        conn.execute(text(f"ALTER TABLE {layer_name} DROP COLUMN Shape"))
                        conn.execute(text(f"EXEC sp_rename '{layer_name}.Shape_Geom', 'Shape', 'COLUMN'"))
                        conn.commit()

                        # Create spatial index
                        self.log(f"   🗂️ Creating spatial index...")
                        spatial_index_sql = f"""
                        CREATE SPATIAL INDEX SIDX_{layer_name}_Shape
                        ON {layer_name}(Shape)
                        USING GEOMETRY_GRID
                        WITH (BOUNDING_BOX = (-180, -90, 180, 90))
                        """
                        conn.execute(text(spatial_index_sql))
                        conn.commit()
                        self.log(f"   ✅ Spatial index created")

                except Exception as e:
                    self.log(f"   ⚠️ Warning: Could not create geometry column or spatial index: {e}", "WARN")

            # Show column mapping changes
            changed_cols = [(orig, new) for orig, new in column_mapping.items()
                           if orig != new and orig != 'geometry']
            if changed_cols:
                self.log(f"   📝 Column name changes:")
                for orig, new in changed_cols[:5]:  # Show first 5
                    self.log(f"      {orig} → {new}")
                if len(changed_cols) > 5:
                    self.log(f"      ... and {len(changed_cols) - 5} more")

            self.log(f"   ✅ Successfully converted {layer_name}")
            return True

        except Exception as e:
            self.log(f"   ❌ Error converting {layer_name}: {e}", "ERROR")
            import traceback
            self.log(f"   Details: {traceback.format_exc()}", "ERROR")
            return False

    def run_conversion(self):
        """Main conversion process"""
        self.log("🚀 Starting FGDB to SQL Server conversion")
        self.log(f"📁 Source: {self.config['fgdb_path']}")
        self.log(f"🎯 Target: {self.config['server']} -> {self.config['database']}")

        # Setup connection
        if not self.setup_connection():
            return False

        # Get layers
        available_layers = self.get_fgdb_layers()
        if not available_layers:
            return False

        # Determine which layers to convert
        if self.config['selected_layers']:
            layers_to_convert = [l for l in self.config['selected_layers'] if l in available_layers]
            missing_layers = [l for l in self.config['selected_layers'] if l not in available_layers]
            if missing_layers:
                self.log(f"⚠️ Selected layers not found: {missing_layers}", "WARN")
        else:
            layers_to_convert = available_layers

        self.log(f"📋 Converting {len(layers_to_convert)} layers: {', '.join(layers_to_convert)}")

        # Convert each layer
        successful = 0
        failed = 0

        for i, layer in enumerate(layers_to_convert, 1):
            self.log(f"\n[{i}/{len(layers_to_convert)}] Processing: {layer}")

            if self.convert_layer(layer):
                successful += 1
            else:
                failed += 1

        # Final summary
        self.log(f"\n" + "="*60)
        self.log(f"📊 CONVERSION SUMMARY")
        self.log(f"="*60)
        self.log(f"✅ Successful: {successful}")
        self.log(f"❌ Failed: {failed}")
        self.log(f"📋 Total: {len(layers_to_convert)}")

        if successful > 0:
            self.log(f"\n🎉 Conversion completed!")
            self.log(f"🔍 Test your data with these SQL queries:")
            self.log(f"   SELECT COUNT(*) FROM {layers_to_convert[0]};")
            self.log(f"   SELECT TOP 5 * FROM {layers_to_convert[0]};")
            self.log(f"   SELECT TOP 5 *, Shape.STAsText() AS WKT FROM {layers_to_convert[0]};")

        return failed == 0

def main():
    """Main function"""
    print("="*60)
    print("    FGDB to SQL Server Converter - FIXED VERSION")
    print("="*60)

    # Show configuration
    print(f"📋 Configuration:")
    print(f"   Server: {CONFIG['server']}")
    print(f"   Database: {CONFIG['database']}")
    print(f"   Username: {CONFIG['username']}")
    print(f"   FGDB: {CONFIG['fgdb_path']}")
    print(f"   Chunk size: {CONFIG['chunk_size']}")
    print(f"   Selected layers: {CONFIG['selected_layers'] or 'All layers'}")
    print()

    # Validate FGDB path
    if not os.path.exists(CONFIG['fgdb_path']):
        print(f"❌ FGDB not found: {CONFIG['fgdb_path']}")
        print(f"💡 Please update CONFIG['fgdb_path'] to the correct location")
        return

    # Create converter and run
    converter = FGDBConverter(CONFIG)
    success = converter.run_conversion()

    if success:
        print(f"\n🏆 All conversions completed successfully!")
    else:
        print(f"\n⚠️ Some conversions failed. Check the log output above.")

if __name__ == "__main__":
    main()

    FGDB to SQL Server Converter - FIXED VERSION
📋 Configuration:
   Server: 100.103.17.32,1433
   Database: SpatialTest
   Username: dbeaver
   FGDB: Z:\Users\brendanhall\GitHub\General_Code\ATFS\FGDB to SQL Server Conversion\esri_ref_data.gdb\esri_ref_data.gdb
   Chunk size: 1000
   Selected layers: All layers

[INFO] 🚀 Starting FGDB to SQL Server conversion
[INFO] 📁 Source: Z:\Users\brendanhall\GitHub\General_Code\ATFS\FGDB to SQL Server Conversion\esri_ref_data.gdb\esri_ref_data.gdb
[INFO] 🎯 Target: 100.103.17.32,1433 -> SpatialTest
[INFO] Setting up SQL Server connection...
[INFO] ✅ Connected to SQL Server
[INFO]    Database: SpatialTest
[INFO]    Version: Microsoft SQL Server 2022 (RTM-GDR) (KB5046861) - ...
[INFO] Reading FGDB: Z:\Users\brendanhall\GitHub\General_Code\ATFS\FGDB to SQL Server Conversion\esri_ref_data.gdb\esri_ref_data.gdb
[INFO] ✅ Found 4 layers: states, fedlandp, park_dtl, dtl_st
[INFO] 📋 Converting 4 layers: states, fedlandp, park_dtl, dtl_st
[INFO] 
[1/4] Proc